In [ ]:
import numpy
from keras.datasets import imdb

from sequence_classifier_comparator import SequenceClassifierComparator
from neural_networks_classifier import NeuralNetworksClassifier
from knn_classifier import KNNClassifier
from svm_classifier import SVMClassifier
from results_reader import ResultsReader
from results_writer import ResultsWriter

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
X = numpy.concatenate((X_train, X_test))
y = numpy.concatenate((y_train, y_test))

In [ ]:
X = X[:1000]
y = y[:1000]

In [ ]:
comparator = SequenceClassifierComparator(ResultsWriter(), ResultsReader())

In [ ]:
nn_classifier = NeuralNetworksClassifier(top_words=top_words)
nn_transformer = nn_classifier.get_transformer()
comparator.add_classifier(nn_classifier, sequence_transformer=nn_transformer)

In [ ]:
svm_classifier = SVMClassifier()
svm_transformer = svm_classifier.get_transformer()
comparator.add_classifier(svm_classifier, sequence_transformer=svm_transformer)

In [ ]:
knn_classifier = KNNClassifier(n_neighbors=1)
knn_transformer = knn_classifier.get_transformer()
comparator.add_classifier(knn_classifier, sequence_transformer=knn_transformer)

In [ ]:
comparator.fit_predict(X, y, rounds=2)

In [ ]:
comparator.show_results()